In [27]:
%pylab inline
import re
import math
import string
from collections import Counter
from __future__ import division

Populating the interactive namespace from numpy and matplotlib


/anaconda3/lib/python3.6/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['sample', 'test', 'product']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
TEXT = open('big.txt').read() #read all the words from big.txt
len(TEXT)

6488666

In [3]:
#we use re package to filter out unnecessary characters, tokenize the words and convert it to lower case
def tokens(text):
    return re.findall('[a-z]+', text.lower()) 

In [28]:
tokens('This is: A test, 1, 2, 3, this is.')

['this', 'is', 'a', 'test', 'this', 'is']

In [29]:
WORDS = tokens(TEXT)
len(WORDS)

1105285

In [30]:
print(WORDS[:10]) #print first few words

['the', 'project', 'gutenberg', 'ebook', 'of', 'the', 'adventures', 'of', 'sherlock', 'holmes']


In [31]:
def sample(bag, n=10):
    return ' '.join(random.choice(bag) for _ in range(n))

In [32]:
sample(WORDS)

'even pale of him to earth and light or in'

In [33]:
Counter(tokens('Is this a test? It is a test!')) #to make up something like a dictionary to store the number of occourences of every word

Counter({'a': 2, 'is': 2, 'it': 1, 'test': 2, 'this': 1})

In [34]:
COUNTS = Counter(WORDS)

print(COUNTS.most_common(10))

[('the', 80030), ('of', 40025), ('and', 38313), ('to', 28766), ('in', 22050), ('a', 21155), ('that', 12512), ('he', 12401), ('was', 11410), ('it', 10681)]


In [35]:
for w in tokens('the rare and neverbeforeseen words'):
    print(w,COUNTS[w])

the 80030
rare 83
and 38313
neverbeforeseen 0
words 460


### Spell Checker
        


In [12]:
def correct(word):
    #Generating all the words with edit distance of 0, 1 & 2
    candidates = (known(edits0(word)) or 
                  known(edits1(word)) or 
                  known(edits2(word)) or 
                  [word])
    return max(candidates, key=COUNTS.get)

In [13]:
def known(words):
    #Return the subset of words that are actually in the dictionary."
    return {w for w in words if w in COUNTS}

def edits0(word): 
    return {word}

def edits2(word):
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

Now for `edits1(word)`: the set of candidate words that are one edit away. For example, given `"wird"`, this would include `"weird"` (inserting an `e`) and `"word"` (replacing a `i` with a `o`), and also `"iwrd"` (transposing `w` and `i`; then `known` can be used to filter this out of the set of final candidates). How could we get them?  One way is to *split* the original word in all possible places, each split forming a *pair* of words, `(a, b)`, before and after the place, and at each place, either delete, transpose, replace, or insert a letter:

<table>
  <tr><td> pairs: <td><tt> Ø+wird <td><tt> w+ird <td><tt> wi+rd <td><tt>wir+d<td><tt>wird+Ø<td><i>Notes:</i><tt> (<i>a</i>, <i>b</i>)</tt> pair</i>
  <tr><td> deletions: <td><tt>Ø+ird<td><tt> w+rd<td><tt> wi+d<td><tt> wir+Ø<td><td><i>Delete first char of b</i>
  <tr><td> transpositions: <td><tt>Ø+iwrd<td><tt> w+rid<td><tt> wi+dr</tt><td><td><td><i>Swap first two chars of b
  <tr><td> replacements: <td><tt>Ø+?ird<td><tt> w+?rd<td><tt> wi+?d<td><tt> wir+?</tt><td><td><i>Replace char at start of b
  <tr><td> insertions: <td><tt>Ø+?+wird<td><tt> w+?+ird<td><tt> wi+?+rd<td><tt> wir+?+d<td><tt> wird+?+Ø</tt><td><i>Insert char between a and b
</table>

In [36]:
def edits1(word):
    pairs      = splits(word)
    deletes    = [a+b[1:]           for (a, b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a, b) in pairs if len(b) > 1]
    replaces   = [a+c+b[1:]         for (a, b) in pairs for c in alphabet if b]
    inserts    = [a+c+b             for (a, b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

def splits(word):
    return [(word[:i], word[i:]) 
            for i in range(len(word)+1)]

alphabet = 'abcdefghijklmnopqrstuvwxyz'

In [37]:
splits('wird')

[('', 'wird'), ('w', 'ird'), ('wi', 'rd'), ('wir', 'd'), ('wird', '')]

In [38]:
print(edits0('wird'))

{'wird'}


In [39]:
print(edits1('wird'))

{'wcrd', 'whrd', 'gwird', 'wirp', 'wjird', 'wirdl', 'wied', 'rwird', 'wiard', 'waird', 'gird', 'wizrd', 'wirxd', 'wirdr', 'wiry', 'wnird', 'wixd', 'oird', 'wsrd', 'wirc', 'wkird', 'wixrd', 'wjrd', 'wirq', 'wirkd', 'iwrd', 'wivrd', 'wfrd', 'wsird', 'wicd', 'wirpd', 'cwird', 'wwird', 'uwird', 'wirz', 'dwird', 'wirj', 'wrid', 'aird', 'hwird', 'word', 'swird', 'fird', 'wdrd', 'wild', 'pird', 'wkrd', 'wirf', 'zwird', 'zird', 'wprd', 'wtird', 'wirdc', 'nird', 'wikrd', 'owird', 'wijrd', 'wifd', 'jwird', 'wijd', 'wisd', 'eird', 'wimd', 'wirg', 'wigd', 'xird', 'wqird', 'wircd', 'wvird', 'twird', 'wirvd', 'bird', 'wipd', 'wirid', 'wirde', 'wnrd', 'wiyd', 'wuird', 'wirtd', 'wiurd', 'wirdt', 'widr', 'wirgd', 'wqrd', 'wira', 'wirdu', 'wirsd', 'wiryd', 'wirl', 'wirdp', 'yird', 'wfird', 'wirs', 'widrd', 'wgird', 'wired', 'wiqd', 'wtrd', 'wirdq', 'iwird', 'wirt', 'ward', 'awird', 'wmird', 'wlird', 'wirdg', 'wirdb', 'wirdh', 'wirdk', 'wirdw', 'wiad', 'wirb', 'wirrd', 'wirdx', 'woird', 'wicrd', 'wirud',

In [40]:
print(len(edits2('wird')))

24254


In [41]:
list(map(correct, tokens('Speling errurs in somethink. Whutever; unusuel misteakes everyware?')))

['spelling',
 'errors',
 'in',
 'something',
 'whatever',
 'unusual',
 'mistakes',
 'everywhere']

Can we make the output prettier than that?

In [42]:
def correct_text(text):
    return re.sub('[a-zA-Z]+', correct_match, text)

def correct_match(match):
    word = match.group()
    return case_of(word)(correct(word.lower()))

def case_of(text):
    return (str.upper if text.isupper() else
            str.lower if text.islower() else
            str.title if text.istitle() else
            str)

In [43]:
correct_text('Speling Errurs IN somethink. Whutever; unusuel misteakes?')

'Spelling Errors IN something. Whatever; unusual mistakes?'

In [44]:
correct_text('Audiance sayzs: tumblr ...')

'Audience says: tumbler ...'

In [45]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(list(counter.values()))
    return lambda x: counter[x]/N

P = pdist(COUNTS)

In [46]:
for w in tokens('"The" is most common word in English'):
    print(P(w), w)

0.0724066643445 the
0.00884296810325 is
0.000821507574969 most
0.00025966153526 common
0.000269613719538 word
0.0199496057578 in
0.000190900989338 english


In [47]:
def Pwords(words):
    return product(P(w) for w in words)

def product(nums):
    result = 1
    for x in nums:
        result *= x
    return result

In [48]:
tests = ['this is a test', 
         'this is a unusual test',
         'this is a neverbeforeseen test']

for test in tests:
    print(Pwords(tokens(test)), test)

2.9833963328e-11 this is a test
8.63747202302e-16 this is a unusual test
0.0 this is a neverbeforeseen test


References:

https://web.stanford.edu/class/cs276/pa/pa2.p

http://web.stanford.edu/~jurafsky/slp3/5.pdf

http://norvig.com/ngrams/spell-errors.txt

http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb

http://norvig.com/big.txt

http://norvig.com/ngrams/count_1w.txt

http://norvig.com/ngrams/